<a href="https://colab.research.google.com/github/easonlai/deepseek_examples/blob/main/Fine_Tuning_DeepSeek_R1_Example_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Install & upgrade necessary libraries.
!pip install --upgrade transformers
!pip install --upgrade bitsandbytes
!pip install --upgrade langchain-community langchain-core
!pip install --upgrade langchain-huggingface
!pip install --upgrade trl

In [2]:
# Display the information about Nvidia GPU devices.
!nvidia-smi

Wed Feb 12 07:38:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Import PyTorch
import torch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
# Import Hugging Face for Login via API
from huggingface_hub import login
# Import the necessary classes from the transformers library
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
# Importing the TrainingArguments class from the transformers library (Used for defining training configurations)
from transformers import TrainingArguments
# Importing the get_peft_model function from the peft library
# (get_peft_model - Used for obtaining parameter-efficient training models)
# (LoraConfig - Used for configuring the Low-Rank Adaptation for parameter-efficient models)
# (TaskType - Defines different task types for model training)
from peft import get_peft_model, LoraConfig, TaskType
# Importing the load_dataset function from the datasets library (Used for loading datasets for training and evaluation)
from datasets import load_dataset
# Import Colab secrets
from google.colab import userdata
# Import LangChain
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
# Importing the wandb library (Used for experiment tracking and logging with Weights and Biases)
import wandb
# Import random library for random generation
import random

In [4]:
# Access saved token secret from Colab.
hugging_face_token = userdata.get('HF_Token_DEV_R')
wnb_token = userdata.get('WNB_Secret_Example_3')

# Login to Hugging Face API with Access Token.
login(hugging_face_token)

# Login to Weights and Biases API with Access Token.
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Example_3',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: easonlai888 (easonlai888-minions-app) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [5]:
# Define the model name, specifying which pre-trained model to use.
# https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-8B
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# Set the maximum sequence length for the model's input.
max_seq_length = 2048

# Configure the quantization settings for efficient model loading and inference.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # Enable 4-bit quantization.
    bnb_4bit_use_double_quant=True, # Use double quantization for improved accuracy.
    bnb_4bit_quant_type="nf4", # Specify the type of 4-bit quantization.
    bnb_4bit_compute_dtype=torch.bfloat16 # Set the computation data type to bfloat16 for optimized performance.
)

# Load the tokenizer for the pre-trained model.
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the pre-trained language model with specific configurations.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, # Set the data type for tensors based on CUDA availability.
    device_map="auto",  # Automatically map the model to available devices (CPU/GPU).
    quantization_config=quantization_config  # Apply the 4-bit quantization for efficient model loading and processing.
)

print("Model and tokenizer loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


In [6]:
# Create a text generation pipeline using the specified model and tokenizer, setting a maximum of 1200 new tokens for the generated text.
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1200)

# Wrap the pipeline in a HuggingFacePipeline object, making it compatible with specific Hugging Face integrations and functionalities.
hf = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [ ]:
# Define a system prompt.
prompt_format = """Question: {question}

Instruction: You are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.

Answer: Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response. Please answer the question in Tranditional Chinese."""

prompt = PromptTemplate.from_template(prompt_format)

chain = prompt | hf

In [ ]:
# Crafting a question for inference.
question = """"過去幾天開始出現流鼻涕、喉嚨痛、打噴嚏和輕微發燒的症狀。
"""

# Print the response to the question.
print(chain.invoke({"question": question}))

Question: "過去幾天開始出現流鼻涕、喉嚨痛、打噴嚏和輕微發燒的症狀。


Instruction: You are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.

Answer: Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response. Please answer the question in Tranditional Chinese. "

首先，患者出现流鼻涕、喉嚨痛、打噴嚏和轻微发烧。这些症状可能提示哪些常见的疾病呢？流鼻涕通常与鼻炎（比如鼻窦炎或鼻腔炎）相关联，而喉嚨痛可能与喉炎或咽喉炎有关。打噴嚏可能是由于鼻腔炎或上呼吸道的炎症导致的。轻微发烧可能是体征的一部分，或者是由感染引起的。接下来，我需要考虑这些症状是否可能由同一个病原体引起，比如流感病毒、COVID-19或者其他呼吸道感染。流鼻涕和喉嚨痛在流感中很常见，同时伴随轻微发烧也是可能的。因此，首先要考虑是否是流感或者其他呼吸道感染。另外，打噴嚏可能与鼻腔炎有关，而喉嚨痛可能与咽喉炎有关，这两种情况可以一起发生，尤其是在鼻炎的情况下。考虑到这些症状，建议进行病因诊断，可能需要做鼻腔和喉咙的检查，或者进行病毒检测。同时，建议患者保持休息，多喝水，使用温和的喷嚏剂来缓解症状。如果症状恶化，比如出现呼吸困难、喉咙严重疼痛或者更严重的发烧，应立即就医。建议患者尽早咨询专业医生，以便进行准确的诊断和治疗。"
"

现在，回答问题，按照上述思考过程，以中文自然表达，避免使用专业术语过多，保持口语化。


---

好的，患者最近几天出现流鼻涕、喉嚨痛、打噴嚏和轻微发烧。首先，我需要考虑这些症状可能的病因。流鼻涕和喉嚨痛可能是鼻炎或咽喉炎的表现。打噴嚏可能与鼻腔炎有关，而轻微发烧可能是体温升高的表现，可能由

In [7]:
# Updated training prompt style by adding </think> tag.
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
Please answer the question in Tranditional Chinese.

### Instruction:
You are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [8]:
# Download and load the dataset using Hugging Face.
# The split parameter specifies that only the first 300 samples from the "train" split are being loaded.
# The trust_remote_code parameter is set to True, allowing the execution of code provided with the dataset.
dataset_full = load_dataset("easonlai/medical-common-diseases-reasoning-SFT", split="train", trust_remote_code=True)

# Define the disease categories and their respective record ranges.
categories = {
    'Common Cold': range(0, 49), # 普通感冒 (Common Cold)
    'Influenza': range(50, 100), # 流感 (Influenza)
    'Gastroenteritis': range(101, 151), # 腸胃炎 (Gastroenteritis)
    'Bronchitis': range(152, 202), # 支氣管炎 (Bronchitis)
    'Sinusitis': range(203, 253), # 鼻竇炎 (Sinusitis)
    'Skin Infections': range(254, 304) # 皮膚感染 (Skin Infections)
}

# Initialize a list to store the selected records.
selected_records = []

# Set the number of records to select.
num_records_to_select = 200

# Calculate the number of records to select from each category proportionally.
records_per_category = num_records_to_select // len(categories)
extra_records = num_records_to_select % len(categories)

for category, record_range in categories.items():
    if category != 'Skin Infections':
        # Randomly select the records.
        selected = random.sample(list(record_range), records_per_category)
        selected_records.extend(selected)
    else:
        # For the last category, add any extra records if needed.
        selected = random.sample(list(record_range), records_per_category + extra_records)
        selected_records.extend(selected)

# Create a new dataset with the selected records.
selected_dataset = dataset_full.select(selected_records)
dataset = selected_dataset

# Displaying the loaded dataset.
dataset

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

(…)edical_common_diseases_sft_tchinese.json:   0%|          | 0.00/222k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/305 [00:00<?, ? examples/s]

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 200
})

In [9]:
# Show the sample entry from the training dataset.
dataset[1]

{'Question': '一名45歲的男性，出現鼻塞、打噴嚏、輕微肌肉酸痛的症狀。這可能是什麼問題？應該怎麼辦？',
 'Complex_CoT': '鼻塞、打噴嚏和輕微肌肉酸痛是普通感冒的常見症狀。感冒病毒通常會影響上呼吸道，引起這些不適。建議患者在家休息，多喝水，使用非處方藥來緩解症狀。如果肌肉酸痛嚴重，可以適當使用止痛藥。總的來說，應該多休息，讓免疫系統對抗病毒，症狀一般會在一週內自行緩解。',
 'Response': '根據描述，這些症狀符合普通感冒的特徵。建議在家休息，多喝水，並使用非處方藥來緩解症狀。如果肌肉酸痛嚴重，可以適當使用止痛藥。'}

In [10]:
# Retrieve and store the End-of-Sequence (EOS) token used by the tokenizer.
# The EOS token is a special token that signals the end of a sequence in text processing.
# It helps the model to understand where a sentence or document ends during tasks like text generation or translation.
EOS_TOKEN = tokenizer.eos_token

# Display the EOS token's value to confirm it has been correctly retrieved from the tokenizer
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [11]:
# Define a function to format prompts from dataset examples.
def formatting_prompts_func(examples):  # Function takes a batch of dataset examples as input.
    inputs = examples["Question"]       # Extracts the common diseases question from the dataset examples.
    cots = examples["Complex_CoT"]      # Extracts the chain-of-thought reasoning for each question.
    outputs = examples["Response"]      # Extracts the model-generated response for each question.

    texts = []  # Initializes an empty list to store the formatted prompts.


    # Iterate over the dataset, formatting each question, reasoning step, and response.
    for input, cot, output in zip(inputs, cots, outputs):
        # Format the question, reasoning, and response using the prompt template
        # Append the End-of-Sequence (EOS) token to indicate the end of the sequence
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        # Add the formatted text to the list
        texts.append(text)

    # Return the newly formatted dataset with a "text" column containing structured prompts.
    return {
        "text": texts,
    }

In [12]:
# Apply the formatting_prompts_func function to each batch of the dataset.
# The map method applies a function to each example in the dataset.
# The batched=True parameter ensures that the function processes the dataset in batches rather than one example at a time.
dataset_finetune = dataset.map(formatting_prompts_func, batched=True)

# Display the first formatted prompt from the "text" column of the fine-tuned dataset.
dataset_finetune["text"][0]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\nPlease answer the question in Tranditional Chinese.\n\n### Instruction:\nYou are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.\n\n### Question:\n一位39歲的女性，近幾天感到鼻塞、咳嗽和輕微肌肉酸痛。這可能是普通感冒嗎？應該怎麼緩解？\n\n### Response:\n<think>\n患者描述的鼻塞、咳嗽和輕微肌肉酸痛可能是普通感冒的症狀。普通感冒通常由病毒引起，治療主要是緩解症狀。建議患者多喝水，保持室內濕潤，使用非處方止痛藥緩解肌肉酸痛。保持適度休息和避免過度勞累，有助於加速康復。\n</think>\n根據描述，這些症狀可能是普通感冒的特徵。建議多喝水、保持室內濕潤，並使用非處方止痛藥來緩解症狀。適當休息，避免過度勞累，以促進康復。<｜end▁of▁sentence｜>'

In [13]:
# Define the configuration for Low-Rank Adaptation (LoRA).
lora_config = LoraConfig(
    r=16,  # Rank of the LoRA matrices, determines the low-rank approximation.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Specific transformer layers to apply LoRA.
    lora_alpha=16,  # Scaling factor to balance the original weights and the adapted weights.
    lora_dropout=0.0,  # Dropout rate for LoRA, used to prevent overfitting.
    bias="none",  # Configuration for bias terms, set to "none" indicating no additional biases are used.
    task_type=TaskType.CAUSAL_LM  # Task type
)

# Apply the Low-Rank Adaptation (LoRA) configuration to the model.
model_lora = get_peft_model(model, lora_config)

In [14]:
tokenizer.padding_side = "right"  # Align padding on the right side

# Initialize the fine-tuning trainer using SFTTrainer from the trl library.
trainer = SFTTrainer(
    model=model_lora,  # Model to be fine-tuned.
    processing_class=tokenizer,  # Tokenizer for processing text inputs.
    train_dataset=dataset_finetune, # Dataset for fine-tuning

    # Training arguments configuration.
    args=TrainingArguments(
        per_device_train_batch_size=1,  # Number of examples processed per device (GPU) at a time.
        gradient_accumulation_steps=4,  # Number of steps to accumulate gradients before updating weights.
        num_train_epochs=1,  # Number of complete passes through the training dataset.
        warmup_steps=5,  # Initial steps with gradually increasing learning rate.
        max_steps=60,  # Total number of training steps (useful for debugging; increase for full fine-tuning).
        learning_rate=2e-4,  # Learning rate for updating model weights (optimized for LoRA fine-tuning).
        fp16=True, # Enable mixed precision training for speed and memory efficiency.
        logging_steps=10,  # Interval at which to log training progress.
        optim="adamw_8bit",  # Use memory-efficient AdamW optimizer in 8-bit mode.
        weight_decay=0.01,  # Regularization parameter to prevent overfitting.
        lr_scheduler_type="linear",  # Learning rate schedule type.
        seed=3407,  # Seed for reproducibility.
        output_dir="outputs",  # Directory to save fine-tuned model checkpoints.
    ),
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
# Start the model fine-tuning process.
trainer_stats = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.610700
20,0.464300
30,0.311200
40,0.287100
50,0.267600
60,0.194700


In [16]:
# Complete the current Weights and Biases (wandb) run and log all remaining data.
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▁▂▃▂▃
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,3448299701231616.0
train/epoch,1.2
train/global_step,60
train/grad_norm,0.58006
train/learning_rate,0
train/loss,0.1947


In [17]:
# Create a text generation pipeline using the specified model and tokenizer, setting a maximum of 1200 new tokens for the generated text.
pipe_lora = pipeline("text-generation", model=model_lora, tokenizer=tokenizer, max_new_tokens=1200)

# Wrap the pipeline in a HuggingFacePipeline object, making it compatible with specific Hugging Face integrations and functionalities.
hf_lora = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

In [18]:
# Define a system prompt.
prompt_format = """Question: {question}

Instruction: You are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.

Answer: Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response. Please answer the question in Tranditional Chinese."""

prompt = PromptTemplate.from_template(prompt_format)

chain_lora = prompt | hf_lora

In [19]:
# Crafting a question for inference.
question = """"過去幾天開始出現流鼻涕、喉嚨痛、打噴嚏和輕微發燒的症狀。
"""

# Print the response to the question.
print(chain_lora.invoke({"question": question}))

Question: "過去幾天開始出現流鼻涕、喉嚨痛、打噴嚏和輕微發燒的症狀。


Instruction: You are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.

Answer: Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response. Please answer the question in Tranditional Chinese..


Answer:
<think>
患者描述的流鼻涕、喉嚨痛、打噴嚏和輕微發燒症狀，可能是普通感冒的症狀。普通感冒通常由病毒引起，治療主要是緩解症狀。建議患者多喝水，保持喉嚨濕潤，可以使用鹽水漱口或喝水。非處方的減充血劑和止痛藥可以幫助緩解喉嚨痛和打噴嚏。保持適度休息有助於加速康復。如果症狀持續或加重，建議就醫。
</think>
這些症狀可能是普通感冒。建議多喝水、保持喉嚨濕潤，使用鹽水漱口或喝水。非處方的減充血劑和止痛藥可以幫助緩解症狀。保持適度休息，促進康復。如果症狀持續或加重，建議就醫。


In [20]:
# Crafting a question for inference.
question = """"過去一天出現腹痛、腹瀉和疲倦的症狀。
"""

# Print the response to the question.
print(chain_lora.invoke({"question": question}))

Question: "過去一天出現腹痛、腹瀉和疲倦的症狀。


Instruction: You are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.

Answer: Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response. Please answer the question in Tranditional Chinese..


Answer:
<think>
患者描述的腹痛、腹瀉和疲倦症狀，可能是腸胃炎的症狀。腸胃炎通常由病毒或細菌感染引起，導致腸胃黏膜的炎症和功能障碍。腹瀉是腸胃炎的常見症狀之一，腹痛和疲倦則可能是腸胃炎的伴隨症狀。建議患者多喝水以防止脫水，並保持充足的休息。同時，避免進食油膩和難消化的食物。如果症狀沒有改善或加重，應尋求醫療幫助。
</think>
這些症狀可能是腸胃炎。建議多喝水、保持充足休息。如果症狀沒有改善或加重，請及時就醫。


In [21]:
# Crafting a question for inference.
question = """"過去幾天開始出現流鼻涕、喉嚨痛、輕微發燒、腹瀉和疲倦的症狀。
"""

# Print the response to the question.
print(chain_lora.invoke({"question": question}))

Question: "過去幾天開始出現流鼻涕、喉嚨痛、輕微發燒、腹瀉和疲倦的症狀。


Instruction: You are an assistant to a general practitioner doctor who is knowledgeable about some common diseases. You can give the answer, but always suggest consulting a human doctor.

Answer: Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response. Please answer the question in Tranditional Chinese..


Answer:
<think>
患者描述的流鼻涕、喉嚨痛、輕微發燒、腹瀉和疲倦症狀，可能是流感或類似感染引起的。流感通常由流感病毒引起，症狀包括發燒、咳嗽、肌肉疼痛和緩慢開始的喉嚨痛。腹瀉和疲倦可能是因為流感導致的減少食欲或體力消耗。建議患者多休息、喝水，並可以使用非處方藥來緩解喉嚨痛和輕微發燒。同時可以使用處方藥來治療腹瀉和減少發燒。若症狀加重或持續，應立即就醫。
</think>
根據描述，這些症狀可能是流感或類似感染。建議多休息、喝水，並使用非處方藥來緩解喉嚨痛和輕微發燒。同時可以使用處方藥來治療腹瀉和減少發燒。如果症狀加重或持續，應立即就醫。
